## Finding all users that have spam flags and deleting them from within salesforce

In [1]:
## Parameters
#hours_range = 24*365*1 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')


salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [2]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Find all users that have block tags from salesforce



In [3]:
# get Salesforce ID for organizations
sf_contact_df = db_s.query_data('SELECT Id, firstname, lastname, nanoHUB_user_ID__c,nanoHUB_account_BLOCKED__c FROM Contact')

# display
sf_contact_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000b1oiAAAQ


{"id":"7505w00000b1oiAAAQ","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-22T11:53:41.000+0000","systemModstamp":"2021-10-22T11:53:41.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b1oiAAAQ","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-22T11:53:41.000+0000","systemModstamp":"2021-10-22T11:53:51.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000b1oiAAAQ","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-22T11:53:41.000+0000","systemModstamp":"2021-10-22T11:53:51.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":14550,"retries":0,"totalProcessingTime":2206}


{"id":"7505w00000b1oiAAAQ","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-22T11:53:41.000+0000","systemModstamp":"2021-10-22T11:54:10.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":262294,"retries":0,"totalProcessingTime":16891}
[Success] Bulk job completed successfully.


,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
0,hubrepo,0035w000031Vsp1AAC,hubrepo,False,998.0
1,nanoHUB,0035w000031Vsp2AAC,support,False,1683.0
2,Grid,0035w000031Vsp3AAC,Statistics,False,1684.0


In [4]:
display(sf_contact_df.tail(3))

,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
247741,Benjamin,0035w00003WNVllAAH,Star,False,1819.0
247742,Harold,0035w00003WNVlmAAH,"Clayton,Sr.",False,8106.0
247743,Sergey,0035w00003WNW5TAAX,Sozykin,False,40187.0


In [5]:
## finding the spam IDs and non-spam IDs
spam_rows = sf_contact_df[sf_contact_df['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("True")]
good_rows = sf_contact_df[~sf_contact_df['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('True')]
display(spam_rows.head(2))
display(good_rows.tail(2))

,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c


,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
247742,Harold,0035w00003WNVlmAAH,"Clayton,Sr.",False,8106.0
247743,Sergey,0035w00003WNW5TAAX,Sozykin,False,40187.0


## delete the spam contacts

In [6]:
%load_ext autoreload
%autoreload 2
# create DB2 to Salesforce API object
db_s = salesforce

db_s.object_id = 'Contact'
#db_s.external_id = 'Id'

In [7]:
# extract only the ids from spam_rows
spam_ids = spam_rows['Id'].to_frame().reset_index().iloc[:,1].to_frame()

display(spam_ids.head(2))
print(type(spam_ids))

,Id


<class 'pandas.core.frame.DataFrame'>


In [8]:
#spam_ids.to_csv('test.csv',index=False,line_terminator='\n',header=False)
spam_ids.iloc[:1]

,Id


In [9]:
# send data to Salesforce
db_s.delete_data(spam_ids)

[Success] Bulk job creation successful. Job ID = 7505w00000b1on8AAA
hello


[Success] CSV upload successful. Job ID = 7505w00000b1on8AAA


[Success] Closing job successful. Job ID = 7505w00000b1on8AAA


In [10]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b1on8AAA',
 'operation': 'delete',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-22T11:54:21.000+0000',
 'systemModstamp': '2021-10-22T11:54:22.000+0000',
 'state': 'UploadComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [11]:
db_s.check_bulk_failed_results()

''